In [149]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maddy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\maddy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\maddy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [150]:
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input, Dense, Embedding, LSTM, Bidirectional, Dropout, Concatenate
from keras.preprocessing.text import Tokenizer
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

In [151]:
df = pd.read_csv('Features_For_Traditional_ML_Techniques.csv')

In [152]:
df.head()

,Unnamed: 0,majority_target,statement,BinaryNumTarget,tweet,followers_count,friends_count,favourites_count,statuses_count,listed_count,...,determiners,conjunctions,dots,exclamation,questions,ampersand,capitals,digits,long_word_freq,short_word_freq
0,0,True,End of eviction moratorium means millions of A...,1.0,@POTUS Biden Blunders - 6 Month Update\n\nInfl...,4262.0,3619.0,34945.0,16423.0,44.0,...,0,0,5,0,1,0,33,3,5,19
1,1,True,End of eviction moratorium means millions of A...,1.0,@S0SickRick @Stairmaster_ @6d6f636869 Not as m...,1393.0,1621.0,31436.0,37184.0,64.0,...,0,2,1,0,0,0,14,0,2,34
2,2,True,End of eviction moratorium means millions of A...,1.0,THE SUPREME COURT is siding with super rich pr...,9.0,84.0,219.0,1184.0,0.0,...,0,1,0,0,0,0,3,0,4,10
3,3,True,End of eviction moratorium means millions of A...,1.0,@POTUS Biden Blunders\n\nBroken campaign promi...,4262.0,3619.0,34945.0,16423.0,44.0,...,0,1,3,0,0,1,6,8,1,30
4,4,True,End of eviction moratorium means millions of A...,1.0,@OhComfy I agree. The confluence of events rig...,70.0,166.0,15282.0,2194.0,0.0,...,0,1,3,0,1,0,11,3,2,19


In [153]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [154]:
df['vadar compound'] = df['statement'].apply(lambda x: analyzer.polarity_scores(x))


In [155]:
df.head(4)

,Unnamed: 0,majority_target,statement,BinaryNumTarget,tweet,followers_count,friends_count,favourites_count,statuses_count,listed_count,...,conjunctions,dots,exclamation,questions,ampersand,capitals,digits,long_word_freq,short_word_freq,vadar compound
0,0,True,End of eviction moratorium means millions of A...,1.0,@POTUS Biden Blunders - 6 Month Update\n\nInfl...,4262.0,3619.0,34945.0,16423.0,44.0,...,0,5,0,1,0,33,3,5,19,"{'neg': 0.275, 'neu': 0.725, 'pos': 0.0, 'comp..."
1,1,True,End of eviction moratorium means millions of A...,1.0,@S0SickRick @Stairmaster_ @6d6f636869 Not as m...,1393.0,1621.0,31436.0,37184.0,64.0,...,2,1,0,0,0,14,0,2,34,"{'neg': 0.275, 'neu': 0.725, 'pos': 0.0, 'comp..."
2,2,True,End of eviction moratorium means millions of A...,1.0,THE SUPREME COURT is siding with super rich pr...,9.0,84.0,219.0,1184.0,0.0,...,1,0,0,0,0,3,0,4,10,"{'neg': 0.275, 'neu': 0.725, 'pos': 0.0, 'comp..."
3,3,True,End of eviction moratorium means millions of A...,1.0,@POTUS Biden Blunders\n\nBroken campaign promi...,4262.0,3619.0,34945.0,16423.0,44.0,...,1,3,0,0,1,6,8,1,30,"{'neg': 0.275, 'neu': 0.725, 'pos': 0.0, 'comp..."


In [159]:
df = pd.concat(
    [df.drop(['Unnamed: 0', 'majority_target', 'vadar compound'], axis=1), 
     df['vadar compound'].apply(pd.Series)], axis=1)
df.head(3)

,statement,BinaryNumTarget,tweet,followers_count,friends_count,favourites_count,statuses_count,listed_count,following,embeddings,...,questions,ampersand,capitals,digits,long_word_freq,short_word_freq,neg,neu,pos,compound
0,End of eviction moratorium means millions of A...,1.0,@POTUS Biden Blunders - 6 Month Update\n\nInfl...,4262.0,3619.0,34945.0,16423.0,44.0,0.0,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]],...,1,0,33,3,5,19,0.275,0.725,0.0,-0.6908
1,End of eviction moratorium means millions of A...,1.0,@S0SickRick @Stairmaster_ @6d6f636869 Not as m...,1393.0,1621.0,31436.0,37184.0,64.0,0.0,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]],...,0,0,14,0,2,34,0.275,0.725,0.0,-0.6908
2,End of eviction moratorium means millions of A...,1.0,THE SUPREME COURT is siding with super rich pr...,9.0,84.0,219.0,1184.0,0.0,0.0,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]],...,0,0,3,0,4,10,0.275,0.725,0.0,-0.6908


In [160]:
df['vadar sentiment'] = df['compound'].apply(lambda x: 'positive' if x >0 else 'neutral' if x==0 else 'negative')


In [161]:
df.head(3)

,statement,BinaryNumTarget,tweet,followers_count,friends_count,favourites_count,statuses_count,listed_count,following,embeddings,...,ampersand,capitals,digits,long_word_freq,short_word_freq,neg,neu,pos,compound,vadar sentiment
0,End of eviction moratorium means millions of A...,1.0,@POTUS Biden Blunders - 6 Month Update\n\nInfl...,4262.0,3619.0,34945.0,16423.0,44.0,0.0,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]],...,0,33,3,5,19,0.275,0.725,0.0,-0.6908,negative
1,End of eviction moratorium means millions of A...,1.0,@S0SickRick @Stairmaster_ @6d6f636869 Not as m...,1393.0,1621.0,31436.0,37184.0,64.0,0.0,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]],...,0,14,0,2,34,0.275,0.725,0.0,-0.6908,negative
2,End of eviction moratorium means millions of A...,1.0,THE SUPREME COURT is siding with super rich pr...,9.0,84.0,219.0,1184.0,0.0,0.0,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]],...,0,3,0,4,10,0.275,0.725,0.0,-0.6908,negative


In [162]:
df['vadar sentiment'].unique()

array(['negative', 'neutral', 'positive'], dtype=object)

In [163]:
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # Remove non-alphanumeric characters
    text = re.sub(r'\W+', ' ', text)
    
    # Tokenization
    tokens = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    
    # Join tokens back into a single string
    preprocessed_text = ' '.join(lemmatized_tokens)
    
    return preprocessed_text

In [164]:
df['preprocessed_text'] = df['statement'].apply(preprocess_text)

In [165]:
df['text'] = df['preprocessed_text'] + df['tweet']
df.drop('statement', axis=1, inplace=True)
df.drop('tweet', axis=1, inplace=True)

In [175]:
label_encoder = LabelEncoder()
df['vadar sentiment'] = label_encoder.fit_transform(df['vadar sentiment'])

In [176]:
X_train, X_test, y_train, y_test = train_test_split(df['preprocessed_text'], df['vadar sentiment'], test_size=0.2, random_state=42)

In [177]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [178]:
max_len = 100
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [179]:
import tensorflow as tf

In [180]:
class AttentionLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(AttentionLayer, self).__init__()

    def build(self, input_shape):
        self.W_q = self.add_weight(shape=(input_shape[-1], 1), initializer='uniform', trainable=True)
        self.W_v = self.add_weight(shape=(input_shape[-1], input_shape[-1]), initializer='uniform', trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, inputs):
        q = tf.matmul(inputs, self.W_q)  # Shape: (batch_size, max_len, 1)
        v = tf.matmul(inputs, self.W_v)  # Shape: (batch_size, max_len, input_shape[-1])
        attn_scores = tf.nn.softmax(q, axis=1)  # Shape: (batch_size, max_len, 1)
        weighted_sum = tf.reduce_sum(v * attn_scores, axis=1)  # Shape: (batch_size, input_shape[-1])
        return weighted_sum

In [181]:
model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, 100, input_length=max_len))
model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)))
model.add(AttentionLayer())  # Use the custom AttentionLayer
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 100, 100)          335500    
                                                                 
 bidirectional_5 (Bidirectio  (None, 100, 256)         234496    
 nal)                                                            
                                                                 
 attention_layer (AttentionL  (None, 256)              65792     
 ayer)                                                           
                                                                 
 dense_8 (Dense)             (None, 128)               32896     
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_9 (Dense)             (None, 1)                

In [182]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32, callbacks=[early_stopping])

Epoch 1/10
3355/3355 [==============================] - 662s 196ms/step - loss: -33625104.0000 - accuracy: 0.5973 - val_loss: -131804176.0000 - val_accuracy: 0.6709
Epoch 2/10
3355/3355 [==============================] - 657s 196ms/step - loss: -471309280.0000 - accuracy: 0.6440 - val_loss: -992332544.0000 - val_accuracy: 0.6334
Epoch 3/10
3355/3355 [==============================] - 638s 190ms/step - loss: -1925332992.0000 - accuracy: 0.6190 - val_loss: -3109052160.0000 - val_accuracy: 0.6929
Epoch 4/10
3355/3355 [==============================] - 648s 193ms/step - loss: -4822876672.0000 - accuracy: 0.6235 - val_loss: -6858515456.0000 - val_accuracy: 0.7057
Epoch 5/10
3355/3355 [==============================] - 647s 193ms/step - loss: -9520418816.0000 - accuracy: 0.6328 - val_loss: -12590837760.0000 - val_accuracy: 0.6965
Epoch 6/10
3355/3355 [==============================] - 633s 189ms/step - loss: -16379051008.0000 - accuracy: 0.6325 - val_loss: -20685297664.0000 - val_accuracy: 0

In [195]:
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)  # Convert probabilities to binary labels (0 or 1)
y_pred = label_encoder.inverse_transform(y_pred).ravel()
y_true = label_encoder.inverse_transform(y_test).ravel()

839/839 [==============================] - 40s 48ms/step


C:\Users\maddy\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_label.py:155: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [191]:
y_true = y_true.ravel()
y_pred = y_pred.ravel()

In [194]:
print("Classification Report:")
print(classification_report(y_true, y_pred, zero_division=1))

Classification Report:
              precision    recall  f1-score   support

    negative       0.95      0.96      0.95      9458
     neutral       0.61      0.96      0.74     11011
    positive       1.00      0.00      0.00      6371

    accuracy                           0.73     26840
   macro avg       0.85      0.64      0.57     26840
weighted avg       0.82      0.73      0.64     26840


Confusion Matrix:
[[ 9066   392     0]
 [  487 10524     0]
 [   10  6361     0]]


In [226]:
import os

# ...

# Save the model with a unique name
model_name = 'Bidirectional_LSTM.h5'
if os.path.exists(model_name):
    os.remove(model_name)
model.save(model_name)

ValueError: Unable to create dataset (name already exists)